In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [2]:
df_csv = pd.read_excel('mexico_train.xlsx')
df_csv

,Id,Category,Topic,Source,Headline,Text,Link
0,1,Fake,Education,El Ruinaversal,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...","RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",http://www.elruinaversal.com/2017/06/10/rae-in...
1,2,Fake,Education,Hay noticia,"La palabra ""haiga"", aceptada por la RAE","La palabra ""haiga"", aceptada por la RAE La Rea...",https://haynoticia.es/la-palabra-haiga-aceptad...
2,3,Fake,Education,El Ruinaversal,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,http://www.elruinaversal.com/2018/05/06/yordi-...
3,4,True,Education,EL UNIVERSAL,UNAM capacitará a maestros para aprobar prueba...,UNAM capacitará a maestros para aprobar prueba...,http://www.eluniversal.com.mx/articulo/nacion/...
4,5,Fake,Education,Lamula,pretenden aprobar libros escolares con conteni...,Alerta: pretenden aprobar libros escolares con...,https://redaccion.lamula.pe/2018/06/19/memoria...
...,...,...,...,...,...,...,...
671,672,Fake,Entertainment,El Dizque,Madonna será la nueva imagen del Chocolate Abu...,Madonna será la nueva imagen del Chocolate Abu...,https://www.eldizque.com/madonna-sera-la-nueva...
672,673,True,Entertainment,Radio Formula,"Filtran material íntimo de Zelina Vega, luchad...","Filtran material íntimo de Zelina Vega, luchad...",http://www.radioformula.com.mx/notas.asp?Idn=7...
673,674,Fake,Entertainment,La Voz Popular,CBS PLANEA GRAN HERMANO EN LA CASA BLANCA CON ...,CBS PLANEA GRAN HERMANO EN LA CASA BLANCA CON ...,http://lavozpopular.com/cbs-planea-gran-herman...
674,675,Fake,Entertainment,El Dizque,TV Azteca anuncia que cambiará su nombre por u...,TV Azteca anuncia que cambiará su nombre por u...,https://www.eldizque.com/tv-azteca-anuncia-que...


In [3]:
df_csv.nunique()

Id          676
Category      2
Topic         9
Source      134
Headline    676
Text        676
Link        676
dtype: int64

In [4]:
df_csv = df_csv.drop_duplicates(subset=['Source'])

In [5]:
df_csv = df_csv[['Source','Link']]
df_csv

,Source,Link
0,El Ruinaversal,http://www.elruinaversal.com/2017/06/10/rae-in...
1,Hay noticia,https://haynoticia.es/la-palabra-haiga-aceptad...
3,EL UNIVERSAL,http://www.eluniversal.com.mx/articulo/nacion/...
4,Lamula,https://redaccion.lamula.pe/2018/06/19/memoria...
5,Heraldo,https://www.heraldo.es/noticias/suplementos/he...
...,...,...
647,El Grafico,http://www.elgrafico.mx/wow/06-03-2017/cuenta-...
658,Caracol Radio,http://caracol.com.co/radio/2013/04/03/entrete...
665,Nacion321,http://www.nacion321.com/elecciones/elena-poni...
667,abc,https://www.abc.es/estilo/gente/abci-donald-tr...


In [6]:
links = []
for link in df_csv.Link:
  links.append(re.findall("https?:\/\/[^\/]+", link)[0])

df_csv['websites'] = links
df_csv = df_csv[['Source', 'websites']]

df_csv

,Source,websites
0,El Ruinaversal,http://www.elruinaversal.com
1,Hay noticia,https://haynoticia.es
3,EL UNIVERSAL,http://www.eluniversal.com.mx
4,Lamula,https://redaccion.lamula.pe
5,Heraldo,https://www.heraldo.es
...,...,...
647,El Grafico,http://www.elgrafico.mx
658,Caracol Radio,http://caracol.com.co
665,Nacion321,http://www.nacion321.com
667,abc,https://www.abc.es


In [8]:
URL = "http://lanic.utexas.edu/la/region/news/"
r = requests.get(URL)
  
soup = BeautifulSoup(r.content, 'lxml') # If this line causes an error, run 'pip install html5lib' or install html5lib


In [9]:
link_list = soup.find('div', attrs = {'id':'directorytext'})

#regional

In [10]:
links = []
for link in link_list.findAll('a'):
  site = {}
  try:
    site['Source'] = link.text.replace('\n', ' ')
    site['websites'] = link['href']

    links.append(site)
    #print(link.text.replace('\n', ' '), link['href'])
  except:
    pass

In [12]:
df_scrap = pd.DataFrame(links)
df_scrap = df_scrap[df_scrap.websites != '#top']
df_scrap = df_scrap.drop_duplicates()
df_scrap

,Source,websites
0,Agencia Latinoamericana de Información,http://www.alainet.org/
1,Asociación Técnica de Diarios Latinoamericanos,http://www.atdl.org/
2,CANANEWS,http://www.cananews.net/
3,Information Services Latin America,http://isla.igc.org/
4,Latin American Newsletters,http://www.latinnews.com/
...,...,...
439,La Tribuna Hispana-USA,http://www.tribunahispanausa.com/
440,La Voz,http://www.azcentral.com/lavoz/
441,Mundo Hispanico,http://www.mundohispanico.com/
442,Prensa Hispana,http://www.prensahispanaaz.com/


In [14]:
df = pd.concat([df_scrap,df_csv])
df

,Source,websites
0,Agencia Latinoamericana de Información,http://www.alainet.org/
1,Asociación Técnica de Diarios Latinoamericanos,http://www.atdl.org/
2,CANANEWS,http://www.cananews.net/
3,Information Services Latin America,http://isla.igc.org/
4,Latin American Newsletters,http://www.latinnews.com/
...,...,...
647,El Grafico,http://www.elgrafico.mx
658,Caracol Radio,http://caracol.com.co
665,Nacion321,http://www.nacion321.com
667,abc,https://www.abc.es


In [15]:
df.Source.nunique()

493

In [16]:
df = df.drop_duplicates('Source')
df

,Source,websites
0,Agencia Latinoamericana de Información,http://www.alainet.org/
1,Asociación Técnica de Diarios Latinoamericanos,http://www.atdl.org/
2,CANANEWS,http://www.cananews.net/
3,Information Services Latin America,http://isla.igc.org/
4,Latin American Newsletters,http://www.latinnews.com/
...,...,...
647,El Grafico,http://www.elgrafico.mx
658,Caracol Radio,http://caracol.com.co
665,Nacion321,http://www.nacion321.com
667,abc,https://www.abc.es


In [ ]:
df.to_csv('Latam_news_websites_0.1.csv')